In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels 
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32


#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

#convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /=255
X_test /=255

# network
model = Sequential()
model.add(Conv2D (32, (3, 3), padding = 'same',
                 input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense (512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# train
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM,
             metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = BATCH_SIZE, 
         epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT, 
         verbose = VERBOSE)
score = model.evaluate(X_test, Y_test, 
                      batch_size = BATCH_SIZE, verbose = VERBOSE)

print("Test score:", score [0])
print('Test accuracy:', score[1])


# save model
model_json = model.to_json()
open('cifer10_architecture.json', 'w').write(model_json)

# save weights
model.save_weights('cifar10_weights.h5', overwrite = True)



X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               4194816   
_______________________________________________

In [ ]:

# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels 
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
NUM_TO_AUGMENT = 5
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

#augmenting
print("Augmenting training set images...")
datagen = ImageDataGenerator(rotation_range = 40, 
                            width_shift_range = 0.2, 
                            height_shift_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip = True, 
                            fill_mode = 'nearest')
xtas, ytas = [], []
for i in range (X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape ((1,) + x.shape) #(1, 3 , 32, 32)
    for x_aug in datagen.flow(x, batch_size=1, 
                             save_to_dir='preview', 
                              save_prefix='cifar', 
                              save_format='jpeg'):
        if num_aug >=NUM_TO_AUGMENT:
            break
            xtax.append(x_aug[0])
            num_aug +=1


#convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /=255
X_test /=255

# network
model = Sequential()
model.add(Conv2D (32, (3, 3), padding = 'same',
                 input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense (512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

#fit the dataget
datagen.fit(X_train)

# train
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM,
             metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = BATCH_SIZE, 
         epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT, 
         verbose = VERBOSE)
score = model.evaluate(X_test, Y_test, 
                      batch_size = BATCH_SIZE, verbose = VERBOSE)

print("Test score:", score [0])
print('Test accuracy:', score[1])


# save model
model_json = model.to_json()
open('cifer10_architecture.json', 'w').write(model_json)

# save weights
model.save_weights('cifar10_weights.h5', overwrite = True)



X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Augmenting training set images...


Ethical and Privacy Concerns for Image Recognition Algorithms

Ethical and privacy concerns for image recognition algorithms are an ongoing debate in technology and societal roles. A facial recognition app has already been created and in use by law enforcement. Clearview AI was created and enables the user to take a picture of an individual and upload it to see public photos of that individual. These photos can include old pictures that have been scraped from social media without the individual's knowledge. This creates a privacy concern. Even if that individual has a private profile if a friend (or anyone) has shared a photo or video of them on their social media that photo can then be used without the individual's knowledge, and in some cases provide their names. This technology was then sold to different law enforcement agencies to use. A cause for another privacy and ethical issue with this app is that every photo that is taken and uploaded to the Clearview AI app is saved to this company's servers, giving the company access to persons of interest, victims, suspects, and much more. If this data is not secured properly, this information could be used against the individuals in malicious ways. There is also a concern with facial rocognition algorithms in that they seem to struggle with persons of color and accurately identifying their different characteristics. This could cause an individual to be misidentified. Larger cities are now using surveillance with the capability of facial recognition and real-time tracking. Individuals are being recorded and monitored without consent further violating their privacy. This technology could be used to targer marginalized/minority groups if misused. Image recognition can also be used to identify well-known locations and can then be used to identify where a specific person may be, further violating an individual's privacy or potential putting them at risk. I believe these concerns will continue to grow as advancements are made in technology without laws and protections in place. 

Reference:
Hill, K. (2020, January 21). The Secretive Company That Might End Privacy as We Know it. 
International New York Times.
https://link.gale.com/apps/doc/A611773186/GIC?u=nhc_main&sid=ebsco&xid=602284b3

Ethical considerations in AI & Maching Learning. Intelegain. (2025, March 20).
https://www.intelegain.com/ethical-considerations-in-ai-machine-learning/

Almeida, D., Shmarko, K., & Lomas, E. (2022), The ethics of Facial Recognition Technologies, surveillance, and accountability in an age of Artificial Intelligence: A comparative analysis of US, EU, and UK Regulatory frameworks. AI and ethics.
https://pmc.ncbi.nlm.nih.gov/articles/PMC8320316/